# Media Campaign Data Cleanup

  Author: Joshua Guillen

___


## Background
___

As the Data Manager for this Media Campaign data project, my key responsibility is to clean and manage two significant datasets: the Device Location and Inventory Viewability datasets. These large datasets are stored in Google Cloud and BigQuery. My role involves meticulously preparing and organizing this raw data, which encompasses tasks such as removing duplicates, handling missing values, and ensuring overall data consistency. Furthermore, I am adding Conversion Revenues and Profit columns to each dataset based on our Inventory and Creatives Conversion Rates.

## Summary
___

Device Location Creative Dataset:

* Removed null values
* Examined unique values across various columns
* Dropped the 'Device_Model' column due to many unknown entries
* Reformulated the Strategy column for clarity
* Created a new 'Conversions_Revenue' and 'Profit' column
* Removed duplicates

Inventory Viewability Dataset:
* Eliminated rows with null values
* Removed the 'Brand_Company_Name' column due to redundancy
* Reformatted the Strategy column to improve readability. This resulted in a new Strategy Device column
* Decided to retain negative values in the 'Measurable_Impressions' column, as they indicated invalid activity
* Created a new 'Conversions_Revenue' and 'Profit' column

## Table of Contents
___

0. Setup / Dependencies
1. Device Location Creative Reporting
    * Initialization
    * Unique Values
    * Exploring Unknown Values
    * Reformatting Strategy Column
    * Adding Conversion Value and Profit Columns
    * Duplicates
    * Exporting Clean CSV
2. Inventory Viewability Reporting
    * Initialization
    * Null Values
    * Dropping Brand Company Name Column
    * Reformatting Strategy Column
    * Negative Measurable Impression Values
    * Adding Conversion Value and Profit Columns
    * Duplicates
    * Exporting Clean CSV


## 0. Setup / Dependencies
---

Importing pandas goodness

In [1]:
import pandas as pd

Importing Google goodness

In [2]:
from google.colab import auth
auth.authenticate_user()

Connecting to my project's data warehouse in BigQuery.

In [3]:
from google.cloud import bigquery
client = bigquery.Client(project='sfda-capstone-project')

Importing Audience Reporting, Device Location Creative Reporting, and Inventory Viewing Data Sets.


In [4]:
QUERY = """
SELECT *
FROM sfda-capstone-project.datasets.device_location_creative_reporting
"""
results = client.query(QUERY)
device_location = results.to_dataframe()


In [5]:
QUERY = """
SELECT *
FROM sfda-capstone-project.datasets.inventory_viewability_reporting
"""
results = client.query(QUERY)
inventory_viewability = results.to_dataframe()

## 1. Device Location Creative Reporting
___

### Initialization

Making copy of raw query for cleaning.

In [6]:
device_location_clean = device_location.copy()

In [7]:
device_location_clean.head()

,Strategy,DMA_Name,Browser,Operating_System,Creative_Size,Device_Type,Device_Make,Device_Model,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost
0,None,None,None,None,None,None,None,None,<NA>,<NA>,<NA>,NaN,NaN
1,Mobile_2P_Custom Intent/Affinity,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),Unknown,1,0,0,0.000974,0.000313
2,Mobile_2P_In Market_Business Services or ISPs,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),Unknown,1,0,0,0.000583,0.000187
3,Mobile_2P_Business Contextual,"Miami-Ft. Lauderdale, Florida",Yandex,Android 8.1,320x50,Smart Phone,BQ (All Models),Unknown,1,0,0,0.000662,0.000213
4,Mobile_2P_Custom Intent/Affinity,"Chicago, Illinois",Google Chrome,Android 8.1,320x50,Smart Phone,BQ (All Models),Unknown,1,0,0,0.001402,0.000450


Removing Nulls on 0th Row

In [8]:
device_location_clean = device_location_clean.drop(0).reset_index(drop=True)

Checking for NULL values

In [9]:
device_location_clean.isna().sum()

Strategy             0
DMA_Name             0
Browser              0
Operating_System     0
Creative_Size        0
Device_Type          0
Device_Make          0
Device_Model         0
Impressions          0
Clicks               0
Total_Conversions    0
Gross_Cost           0
Net_Cost             0
dtype: int64

Dataframe Info

In [10]:
device_location_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464433 entries, 0 to 464432
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Strategy           464433 non-null  object 
 1   DMA_Name           464433 non-null  object 
 2   Browser            464433 non-null  object 
 3   Operating_System   464433 non-null  object 
 4   Creative_Size      464433 non-null  object 
 5   Device_Type        464433 non-null  object 
 6   Device_Make        464433 non-null  object 
 7   Device_Model       464433 non-null  object 
 8   Impressions        464433 non-null  Int64  
 9   Clicks             464433 non-null  Int64  
 10  Total_Conversions  464433 non-null  Int64  
 11  Gross_Cost         464433 non-null  float64
 12  Net_Cost           464433 non-null  float64
dtypes: Int64(3), float64(2), object(8)
memory usage: 47.4+ MB


In [11]:
device_location_clean.describe(include="all")

,Strategy,DMA_Name,Browser,Operating_System,Creative_Size,Device_Type,Device_Make,Device_Model,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost
count,464433,464433,464433,464433,464433,464433,464433,464433,464433.0,464433.0,464433.0,464433.000000,464433.000000
unique,6,121,4,71,3,2,100,1891,<NA>,<NA>,<NA>,NaN,NaN
top,Mobile_RON,"Chicago, Illinois",Google Chrome,Android 13.0,320x50,Smart Phone,Samsung (All Models),Unknown,<NA>,<NA>,<NA>,NaN,NaN
freq,116148,28992,424958,118923,194390,406553,225662,67488,<NA>,<NA>,<NA>,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153.023939,0.072351,0.002915,0.263574,0.084607
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3474.842331,1.316353,0.146267,5.918069,1.899700
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.001619,0.000520
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.0,0.0,0.005172,0.001660
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,0.0,0.0,0.024769,0.007951


### Unique Values

Looking at unique values in Strategy category. It seems every audience is a 2nd party mobile custom audience, except for RON, which means 'Run of Network', which we could categorize as a custom audience. These labels would benefit from some better formatting such as removing the 'Mobile 2P' and the underscores.

In [12]:
device_location_clean.Strategy.unique()

array(['Mobile_2P_Custom Intent/Affinity',
       'Mobile_2P_In Market_Business Services or ISPs',
       'Mobile_2P_Business Contextual', 'Mobile_RON',
       'Mobile_2P_Pixel Converters_Optimized Targeting',
       'Mobile_2P_Affinity Categories'], dtype=object)

Looking at unique values in Device Model. Note: One of the common values is 'Unknown' which requires some more exploration to determine what to do that data.

In [13]:
device_location_clean.Device_Model.unique()

array(['Unknown', 'bq AQUARIS X', 'LG V60 ThinQ 5G (LM-V600)', ...,
       'General Mobile GM 5 PLUS D', 'General Mobile GM8 GO',
       'General Mobile GM 8 D'], dtype=object)

Unique values in device type.

In [14]:
device_location_clean.Device_Type.unique()

array(['Smart Phone', 'Tablet'], dtype=object)

Unique values in Browser

In [15]:
device_location_clean.iloc[:, 2].unique()

array(['Google Chrome', 'Yandex', 'Microsoft Edge', 'Android Browser'],
      dtype=object)

Unique values in OS

In [16]:
device_location_clean.iloc[:, 3].unique()

array(['Android 4.4', 'Android 8.1', 'Android 10.0', 'Android 11.0',
       'Android 4.2', 'Android 5.0', 'Android 9.0', 'WindowsPhone 10.0',
       'Android', 'Android 5.1', 'Android 6.0', 'Android 7.0',
       'Android 7.1', 'Android 8.0', 'Android 12.0', 'Android 13.0',
       'Android 4.3', 'iOS', 'iOS 7.0', 'iOS 9.0', 'iOS 9.2', 'iOS 10.0',
       'iOS 10.1', 'iOS 10.2', 'iOS 10.3', 'iOS 11.0', 'iOS 11.1',
       'iOS 11.2', 'iOS 11.3', 'iOS 11.4', 'iOS 12.0', 'iOS 12.1',
       'iOS 12.2', 'iOS 12.3', 'iOS 12.4', 'iOS 12.5', 'iOS 13.0',
       'iOS 13.1', 'iOS 13.2', 'iOS 13.3', 'iOS 13.4', 'iOS 13.5',
       'iOS 13.6', 'iOS 13.7', 'iOS 14.0', 'iOS 14.1', 'iOS 14.2',
       'iOS 14.3', 'iOS 14.4', 'iOS 14.5', 'iOS 14.6', 'iOS 14.7',
       'iOS 14.8', 'iOS 15.0', 'iOS 15.1', 'iOS 15.2', 'iOS 15.3',
       'iOS 15.4', 'iOS 15.5', 'iOS 15.6', 'iOS 15.7', 'iOS 16.0',
       'iOS 16.1', 'iOS 16.2', 'iOS 16.3', 'iOS 16.4', 'iOS 16.5',
       'Android 4.0', 'Android 4.1', 'Android 2.3

Unique Creative Sizes

In [17]:
device_location_clean.iloc[:,4].unique()

array(['300x250', '320x50', '300x50'], dtype=object)

### Exploring Unknown Values

Filtering for unknown devices

In [18]:
unknown_devices_filter = device_location_clean.loc[:, 'Device_Model'] == "Unknown"

In [19]:
unknown_devices = device_location_clean.loc[unknown_devices_filter, :]

Over 67k unknown entries!

In [20]:
unknown_devices.shape

(67488, 13)

There are certain Device Makes that do not have unknown values.

In [21]:
list(unknown_devices.Device_Make.unique()) == list(device_location_clean.Device_Make.unique())

False

In [22]:
unknown_devices.Device_Make.unique()

array(['BQ (All Models)', 'HP (All Models)', 'LG (All Models)',
       'Blu (All Models)', 'HTC (All Models)', 'IMO (All Models)',
       'NEC (All Models)', 'NGM (All Models)', 'RCA (All Models)',
       'TCL (All Models)', 'THL (All Models)', 'ZTE (All Models)',
       'Acer (All Models)', 'Asus (All Models)', 'Dell (All Models)',
       'Eton (All Models)', 'LAVA (All Models)', 'OPPO (All Models)',
       'Palm (All Models)', 'Sony (All Models)', 'Vivo (All Models)',
       'Wiko (All Models)', 'Advan (All Models)', 'Azumi (All Models)',
       'Chuwi (All Models)', 'Cubot (All Models)', 'Haier (All Models)',
       'Intel (All Models)', 'Intex (All Models)', 'Lanix (All Models)',
       'Meizu (All Models)', 'Nokia (All Models)', 'Sharp (All Models)',
       'Sonim (All Models)', 'Tecno (All Models)', 'Vizio (All Models)',
       'iBall (All Models)', 'Amazon (All Models)', 'Archos (All Models)',
       'Doogee (All Models)', 'Explay (All Models)',
       'Gionee (All Models)', 'Go

Only 8 makes have no unknown models in their rows.

In [23]:
unknown_makes = list(unknown_devices.Device_Make.unique())
all_makes = list(device_location_clean.Device_Make.unique())
not_in_unknown = []

# find the Device Makes that do not have any unknown models
for i in range(len(all_makes)):
    if (all_makes[i] not in unknown_makes):
        not_in_unknown.append(all_makes[i])

not_in_unknown

['HDC (All Models)',
 'Apple (All Models)',
 'Itoos (All Models)',
 'DoCoMo (All Models)',
 'Yifang (All Models)',
 'Pioneer (All Models)',
 'Vodafone (All Models)',
 'SonyEricsson (All Models)']

Since the Device Make column has a significant amount of unknown values (67,488), all of which can not be filled in with existing data, it should be dropped entirely since any analysis of the Device Makes would be skewed and not representative of the true audience.

In [24]:
device_location_clean = device_location_clean.drop('Device_Model', axis=1)
device_location_clean.head()

,Strategy,DMA_Name,Browser,Operating_System,Creative_Size,Device_Type,Device_Make,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost
0,Mobile_2P_Custom Intent/Affinity,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),1,0,0,0.000974,0.000313
1,Mobile_2P_In Market_Business Services or ISPs,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),1,0,0,0.000583,0.000187
2,Mobile_2P_Business Contextual,"Miami-Ft. Lauderdale, Florida",Yandex,Android 8.1,320x50,Smart Phone,BQ (All Models),1,0,0,0.000662,0.000213
3,Mobile_2P_Custom Intent/Affinity,"Chicago, Illinois",Google Chrome,Android 8.1,320x50,Smart Phone,BQ (All Models),1,0,0,0.001402,0.000450
4,Mobile_2P_Custom Intent/Affinity,"Detroit, Michigan",Google Chrome,Android 8.1,300x50,Smart Phone,BQ (All Models),1,0,0,0.000703,0.000226


### Reformatting Strategy Column

The "Mobile" and "2P" labels in the Strategy are redundant since it applies to every entry. Therefore we will update every value under this category with only their specific strategy.

In [25]:
device_location_clean = device_location_clean.replace(to_replace='Mobile_2P_Custom Intent/Affinity', value='Custom Intent/Affinity')
device_location_clean = device_location_clean.replace(to_replace='Mobile_2P_In Market_Business Services or ISPs', value='In Market, Business Services or ISPs')
device_location_clean = device_location_clean.replace(to_replace='Mobile_2P_Business Contextual', value='Business Contextual')
device_location_clean = device_location_clean.replace(to_replace='Mobile_RON', value='Run of Network')
device_location_clean = device_location_clean.replace(to_replace='Mobile_2P_Pixel Converters_Optimized Targeting', value='Pixel Converters, Optimized Targeting')
device_location_clean = device_location_clean.replace(to_replace='Mobile_2P_Affinity Categories', value='Affinity Categories')

If there were more than 6 distinct values, I would have considered a creating a more sophisticated method of updating these values. This is fine for now.

In [26]:
device_location_clean.head(3)

,Strategy,DMA_Name,Browser,Operating_System,Creative_Size,Device_Type,Device_Make,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost
0,Custom Intent/Affinity,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),1,0,0,0.000974,0.000313
1,"In Market, Business Services or ISPs","Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),1,0,0,0.000583,0.000187
2,Business Contextual,"Miami-Ft. Lauderdale, Florida",Yandex,Android 8.1,320x50,Smart Phone,BQ (All Models),1,0,0,0.000662,0.000213


### Adding Conversion Value and Profit Columns

I can create a new column, "Conversions_Revenue" by multiplying each row's total conversions by the conversion value for Mobile Creatives. From our [guidelines](https://docs.google.com/presentation/d/1zieAer3-1WF3rS6wtzrh0eO-KtSYHl0JVq19Q-iwp-Y/edit#slide=id.g24db77693a5_0_457), the conversion value for our Mobile Creatives is $120.47.

In [27]:
device_location_clean.loc[:, 'Conversions_Revenue'] = device_location_clean.Total_Conversions * 120.47

Double-checking to see if it worked shows our top Conversion revenue as $3,855.04 from 32 total conversions.

In [28]:
device_location_clean.sort_values('Conversions_Revenue', ascending=False).head(3)

,Strategy,DMA_Name,Browser,Operating_System,Creative_Size,Device_Type,Device_Make,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Conversions_Revenue
57455,"Pixel Converters, Optimized Targeting","Grand Rapids-Kalamazoo, Michigan",Google Chrome,Android 10.0,320x50,Smart Phone,Wiko (All Models),89539,24,32,149.118518,47.867044,3855.04
57434,"Pixel Converters, Optimized Targeting","Chicago, Illinois",Google Chrome,Android 10.0,320x50,Smart Phone,Wiko (All Models),558213,246,28,1065.355671,341.979170,3373.16
83888,"Pixel Converters, Optimized Targeting","Atlanta, Georgia",Google Chrome,iOS 16.5,300x250,Smart Phone,Apple (All Models),168619,47,25,270.247682,86.749506,3011.75


Now for profit. We'll calculate it using Conversions Revenue - Gross Cost

In [29]:
device_location_clean.loc[:, 'Profit'] = device_location_clean.Conversions_Revenue - device_location_clean.Gross_Cost

Very nice!

In [30]:
device_location_clean.sort_values('Profit', ascending=False).head(3)

,Strategy,DMA_Name,Browser,Operating_System,Creative_Size,Device_Type,Device_Make,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Conversions_Revenue,Profit
57455,"Pixel Converters, Optimized Targeting","Grand Rapids-Kalamazoo, Michigan",Google Chrome,Android 10.0,320x50,Smart Phone,Wiko (All Models),89539,24,32,149.118518,47.867044,3855.04,3705.921482
320665,Custom Intent/Affinity,"Grand Rapids-Kalamazoo, Michigan",Google Chrome,Android 13.0,320x50,Smart Phone,Samsung (All Models),3988,5,23,9.666359,3.102901,2770.81,2761.143641
83888,"Pixel Converters, Optimized Targeting","Atlanta, Georgia",Google Chrome,iOS 16.5,300x250,Smart Phone,Apple (All Models),168619,47,25,270.247682,86.749506,3011.75,2741.502318


### Duplicates

There's 193 duplicates!

In [31]:
device_location_clean.duplicated(keep='last').sum()

193

In [32]:
device_location_clean = device_location_clean.drop_duplicates()

Back to normal!

In [33]:
device_location_clean.duplicated(keep='last').sum()

0

### Exporting Clean CSV

Our Device Location dataset is satisfactorily clean for my liking. Time to export it!

In [34]:
device_location_clean.to_csv('device_location_clean.csv')

## 2. Inventory Viewability Reporting
___

### Initialization

Making copy of inventory viewability dataset so as to not alter our raw data.

In [35]:
inventory_viewability_clean = inventory_viewability.copy()
inventory_viewability_clean.head()

,Brand_Company_Name,Strategy__,App_URL,Exchange,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Viewable_Impressions,Measurable_Impressions
0,Choose your advertiser name,Desktop_2P_Affinity Categories,al.com,Taboola,2401,0,0,3.106056,0.997044,1551,2122
1,Choose your advertiser name,Desktop_2P_Affinity Categories,carscoops.com,Taboola,148,0,0,0.527778,0.169417,37,126
2,Choose your advertiser name,Desktop_2P_Affinity Categories,cleveland.com,Taboola,180,0,0,0.235679,0.075653,97,151
3,Choose your advertiser name,Desktop_2P_Affinity Categories,doviz.com,Adform,19,0,0,0.003110,0.000998,12,21
4,Choose your advertiser name,Desktop_2P_Affinity Categories,e.walla.co.il,Taboola,26,0,0,0.032122,0.010311,7,27


Data has over a million rows (1,037,468) across 11 rows!

In [36]:
inventory_viewability_clean.shape

(1037468, 11)

Inventory Viewability dataset info

In [37]:
inventory_viewability_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037468 entries, 0 to 1037467
Data columns (total 11 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Brand_Company_Name      1037466 non-null  object 
 1   Strategy__              1037466 non-null  object 
 2   App_URL                 1037466 non-null  object 
 3   Exchange                1037466 non-null  object 
 4   Impressions             1037466 non-null  Int64  
 5   Clicks                  1037466 non-null  Int64  
 6   Total_Conversions       1037466 non-null  Int64  
 7   Gross_Cost              1037466 non-null  float64
 8   Net_Cost                1037466 non-null  float64
 9   Viewable_Impressions    1037466 non-null  Int64  
 10  Measurable_Impressions  1037466 non-null  Int64  
dtypes: Int64(5), float64(2), object(4)
memory usage: 92.0+ MB


In [38]:
inventory_viewability_clean.describe(include='all')

,Brand_Company_Name,Strategy__,App_URL,Exchange,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Viewable_Impressions,Measurable_Impressions
count,1037466,1037466,1037466,1037466,1037466.0,1037466.0,1037466.0,1.037466e+06,1.037466e+06,1037466.0,1037466.0
unique,1,12,50284,25,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
top,Choose your advertiser name,Mobile_2P_Pixel Converters_Optimized Targeting,dailymail.co.uk/news,Google Ad Manager,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
freq,1037466,147137,197,255181,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
mean,NaN,NaN,NaN,NaN,184.121144,0.042055,0.005111,2.688153e-01,8.628972e-02,83.242302,154.991127
std,NaN,NaN,NaN,NaN,8588.326978,1.675789,0.351782,1.081095e+01,3.470313e+00,4839.753191,7438.227202
min,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,-2168.0
25%,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.648000e-03,5.290080e-04,0.0,1.0
50%,NaN,NaN,NaN,NaN,4.0,0.0,0.0,4.751000e-03,1.525071e-03,2.0,3.0
75%,NaN,NaN,NaN,NaN,15.0,0.0,0.0,2.035200e-02,6.532992e-03,6.0,12.0


Some observations include:


*   Brand_Company_Name column has only one unique value
*   Strategy column format is hard to read
*   Measureable_Impressions column has negative values




### Null Values

Checking for NULL values. Seems like 2 rows in each category has a NULL value.

In [39]:
inventory_viewability_clean.isna().sum()

Brand_Company_Name        2
Strategy__                2
App_URL                   2
Exchange                  2
Impressions               2
Clicks                    2
Total_Conversions         2
Gross_Cost                2
Net_Cost                  2
Viewable_Impressions      2
Measurable_Impressions    2
dtype: int64

Exploring the nulls of the Net_Cost category, it reveals that the nulls are shared across each category across two rows.

In [40]:
null_filter = inventory_viewability_clean.loc[:, "Net_Cost"].isna()
inventory_viewability_clean.loc[null_filter, :]

,Brand_Company_Name,Strategy__,App_URL,Exchange,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Viewable_Impressions,Measurable_Impressions
623,None,None,None,None,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
16399,None,None,None,None,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>


Let's drop these rows!

In [41]:
inventory_viewability_clean = inventory_viewability_clean.dropna().reset_index(drop=True)

Cheeky lil double check.

In [42]:
inventory_viewability_clean.isna().sum()

Brand_Company_Name        0
Strategy__                0
App_URL                   0
Exchange                  0
Impressions               0
Clicks                    0
Total_Conversions         0
Gross_Cost                0
Net_Cost                  0
Viewable_Impressions      0
Measurable_Impressions    0
dtype: int64

Perfect!

### Dropping Brand Company Name Column

Brand_Company_Name only has one value, "Choose your advertiser name".

In [43]:
inventory_viewability_clean.Brand_Company_Name.unique()

array(['Choose your advertiser name'], dtype=object)

Since this is not helpful towards our data analysis, I will drop it.

In [44]:
inventory_viewability_clean = inventory_viewability_clean.drop('Brand_Company_Name', axis=1)
inventory_viewability_clean.head(1)

,Strategy__,App_URL,Exchange,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Viewable_Impressions,Measurable_Impressions
0,Desktop_2P_Affinity Categories,al.com,Taboola,2401,0,0,3.106056,0.997044,1551,2122


### Reformatting Strategy Column

The error below demonstrates that not only do the values of the Strategy column need to be reformatted, but the column name itself.

In [45]:
inventory_viewability_clean.Strategy.unique()

AttributeError: ignored

Cheeky lil rename.

In [46]:
inventory_viewability_clean = inventory_viewability_clean.rename(columns={'Strategy__':"Strategy"})

Let's try to see the unique values now. This time we have not only mobile, but desktop devices as part of our strategy values.

In [47]:
inventory_viewability_clean.Strategy.unique()

array(['Desktop_2P_Affinity Categories', 'Desktop_2P_Business Contextual',
       'Desktop_2P_Custom Intent/Affinity',
       'Desktop_2P_In Market_Business Services or ISPs',
       'Desktop_2P_Pixel Converters_Optimized Targeting', 'Desktop_RON',
       'Mobile_2P_Affinity Categories',
       'Mobile_2P_Custom Intent/Affinity',
       'Mobile_2P_In Market_Business Services or ISPs',
       'Mobile_2P_Pixel Converters_Optimized Targeting', 'Mobile_RON',
       'Mobile_2P_Business Contextual'], dtype=object)

This reformatting requires a bit more sophistication, I will start by seperating the first string by the '_' delimiter and expanding it into it's own column.

In [48]:
new_columns = inventory_viewability_clean.Strategy.str.split("_", n=1, expand=True)
new_columns.head(1)

,0,1
0,Desktop,2P_Affinity Categories


In [49]:
inventory_viewability_clean['Strategy_Device'] = new_columns[0]

In [50]:
inventory_viewability_clean.head(1)

,Strategy,App_URL,Exchange,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Viewable_Impressions,Measurable_Impressions,Strategy_Device
0,Desktop_2P_Affinity Categories,al.com,Taboola,2401,0,0,3.106056,0.997044,1551,2122,Desktop


I will drop our original Strategy column to make room for a new Strategy column with our new format.

In [51]:
inventory_viewability_clean = inventory_viewability_clean.drop('Strategy', axis=1)

Adding our new column.

In [52]:
inventory_viewability_clean['Strategy'] = new_columns[1]
inventory_viewability_clean.Strategy.unique()

array(['2P_Affinity Categories', '2P_Business Contextual',
       '2P_Custom Intent/Affinity',
       '2P_In Market_Business Services or ISPs',
       '2P_Pixel Converters_Optimized Targeting', 'RON'], dtype=object)

We aren't done yet. I want to remove the "2P" if the value has it, since it's redundant. And replace underscores with a comma and whitespace.

In [53]:
inventory_viewability_clean.Strategy = inventory_viewability_clean.Strategy.str.replace('2P_', '')
inventory_viewability_clean.Strategy = inventory_viewability_clean.Strategy.str.replace('_', ', ')

PERFECT!

In [54]:
inventory_viewability_clean.Strategy.unique()

array(['Affinity Categories', 'Business Contextual',
       'Custom Intent/Affinity', 'In Market, Business Services or ISPs',
       'Pixel Converters, Optimized Targeting', 'RON'], dtype=object)

### Negative Measureable Impression Values

It is unusual why there are negative values for our Measureable Impressions column. This calls for an investigation.

In [55]:
negative_impressions_filter = inventory_viewability_clean.loc[:, "Measurable_Impressions"] < 0
negative_impressions = inventory_viewability_clean.loc[negative_impressions_filter, :]
negative_impressions.head()

,App_URL,Exchange,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Viewable_Impressions,Measurable_Impressions,Strategy_Device,Strategy
9280,canarias7.es,OpenX,0,0,0,0.000000,0.000000,0,-1,Desktop,Affinity Categories
9285,centralillinoisproud.com,OpenX,6,0,0,0.003973,0.001275,1,-3,Desktop,Affinity Categories
9322,fox56news.com,OpenX,0,0,0,0.000000,0.000000,0,-1,Desktop,Affinity Categories
9323,fox5sandiego.com,OpenX,4,0,0,0.004913,0.001577,0,-5,Desktop,Affinity Categories
9337,housemixblog.com,OpenX,0,0,0,0.000000,0.000000,0,-1,Desktop,Affinity Categories


We have 1,612 rows with negative values. Compared to other million rows of our dataset, this seems insignificant, but I still think it's worth understanding and perhaps transforming.

In [56]:
negative_impressions.shape

(1612, 11)

According to this [article](https://support.google.com/admanager/thread/177613700/why-is-ad-manager-showing-negative-impressions-in-reports?hl=en), negative impressions result from invalid activity. I'll leave this data as it is since understanding which ads have more invalid traffic than others might be an interesting aspect to analyze in order to understand which ads receive this behavior more than others.

### Adding Conversion Value and Profit Columns

Since this dataset has both Desktop and Mobile Inventory, there are two different conversion rates I need to include when calculating Conversion Revenue. I'm going to go about this by making two Data Frames, one for Desktop Inventory and another for the Mobile Inventory, calculating their conversion revenue, and then combining them back into one dataset. Let's start with the Desktop Inventory, which has a [Conversion Rate](https://docs.google.com/presentation/d/1zieAer3-1WF3rS6wtzrh0eO-KtSYHl0JVq19Q-iwp-Y/edit#slide=id.g24db77693a5_0_457) of $46.42.

In [57]:
desktop_filter = inventory_viewability_clean.Strategy_Device == 'Desktop'
desktop_inventory = inventory_viewability_clean.loc[desktop_filter, :].copy() #need a copy here
desktop_inventory.loc[:, 'Conversions_Revenue'] = desktop_inventory.Total_Conversions * 46.42
desktop_inventory.sort_values('Conversions_Revenue', ascending=False).head(3)

,App_URL,Exchange,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Viewable_Impressions,Measurable_Impressions,Strategy_Device,Strategy,Conversions_Revenue
734152,msn.com,Xandr - Monetize SSP (AppNexus),4154911,261,154,3037.023912,974.884676,2338732,3445626,Desktop,"Pixel Converters, Optimized Targeting",7148.68
932323,ebay.com,Google Ad Manager,1100942,129,147,3228.161720,1036.239912,629973,1041225,Desktop,"Pixel Converters, Optimized Targeting",6823.74
188544,mail.yahoo.com,PubMatic,1956642,194,106,3807.329578,1222.152795,981461,1731673,Desktop,"Pixel Converters, Optimized Targeting",4920.52


Now for our Mobile Inventory! This time, the conversion rate is $54.23.

In [58]:
mobile_filter = inventory_viewability_clean.Strategy_Device == 'Mobile'
mobile_inventory = inventory_viewability_clean.loc[mobile_filter, :].copy() #need a copy here
mobile_inventory.loc[:, 'Conversions_Revenue'] = mobile_inventory.Total_Conversions * 54.23
mobile_inventory.sort_values('Conversions_Revenue', ascending=False).head(3)

,App_URL,Exchange,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Viewable_Impressions,Measurable_Impressions,Strategy_Device,Strategy,Conversions_Revenue
466782,unsophisticook.com,Google Ad Manager,130,0,23,0.283107,0.090877,72,120,Mobile,Custom Intent/Affinity,1247.29
793650,miamiherald.com,Magnite DV+,7320,9,22,14.447019,4.637493,4020,6692,Mobile,"Pixel Converters, Optimized Targeting",1193.06
943618,the-sun.com,Google Ad Manager,247106,290,19,777.213887,249.485658,159618,233150,Mobile,"Pixel Converters, Optimized Targeting",1030.37


Now we have to combine these two copies back into one dataset.

In [59]:
inventory_viewability_clean = pd.concat([desktop_inventory, mobile_inventory]).reset_index(drop=True)

And just to make sure we didn't destroy everything...

In [60]:
inventory_viewability_clean.shape

(1037466, 12)

In [61]:
inventory_viewability_clean.Strategy_Device.unique()

array(['Desktop', 'Mobile'], dtype=object)

Looks good! Now for profit. Again we're using the same calculation as last time.

In [62]:
inventory_viewability_clean.loc[:, 'Profit'] = inventory_viewability_clean.Conversions_Revenue - inventory_viewability_clean.Gross_Cost

Looks great to me!

In [63]:
inventory_viewability_clean.sort_values('Profit', ascending=False).head(3)

,App_URL,Exchange,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Viewable_Impressions,Measurable_Impressions,Strategy_Device,Strategy,Conversions_Revenue,Profit
382549,msn.com,Xandr - Monetize SSP (AppNexus),4154911,261,154,3037.023912,974.884676,2338732,3445626,Desktop,"Pixel Converters, Optimized Targeting",7148.68,4111.656088
491726,ebay.com,Google Ad Manager,1100942,129,147,3228.161720,1036.239912,629973,1041225,Desktop,"Pixel Converters, Optimized Targeting",6823.74,3595.57828
141998,mail.yahoo.com,BidSwitch,1323451,97,97,1621.901346,520.630332,737990,1177859,Desktop,"Pixel Converters, Optimized Targeting",4502.74,2880.838654


### Duplicates

No dupes! Makes my job easier.

In [64]:
inventory_viewability_clean.duplicated(keep='last').sum()

0

### Exporting Clean CSV

Before we export this data, I'll reindex the columns to have a better data sequence.

In [65]:
inventory_viewability_clean = inventory_viewability_clean.reindex(columns=['Strategy_Device', 'Strategy', 'App_URL',
      'Exchange', 'Impressions', 'Clicks', 'Total_Conversions',
       'Gross_Cost', 'Net_Cost', 'Viewable_Impressions',
       'Measurable_Impressions', 'Conversions_Revenue', 'Profit'])
print(list(inventory_viewability_clean.columns))

['Strategy_Device', 'Strategy', 'App_URL', 'Exchange', 'Impressions', 'Clicks', 'Total_Conversions', 'Gross_Cost', 'Net_Cost', 'Viewable_Impressions', 'Measurable_Impressions', 'Conversions_Revenue', 'Profit']


Our Inventory Viewability dataset is satisfactorily clean for my liking. Time to export it!

In [66]:
inventory_viewability_clean.to_csv('inventory_viewability_clean.csv')